In [29]:
import os
import matplotlib.pyplot as plt
import pandas
import numpy
import matplotlib
import pandas as pd
from matplotlib.axis import Axis
from matplotlib.figure import Figure
from matplotlib.patches import Rectangle

from env import *
from visualiser import *
from main import *
from constants import *
# import xlsxwriter
import csv
%matplotlib inline

In [30]:
SENSOR_RANGE = 10

n=range(2, 11)
ENV_SIZE=1000
XL=500
YL=100

GOAL_X=500
GOAL_Y=900
MAX_T=50000

In [31]:
def calcDF(n):
    return [(SENSOR_RANGE+ROBOT_RADIUS)*(i-(n-1)/2) for i in range(n)] * n, [(SENSOR_RANGE+ROBOT_RADIUS)*(i-(n-1)/2) for i in range(n)for _ in range(n) ]

In [32]:
def plot_trajectory(env, fig:Figure = None, ax:Axis=None):

	if fig is None and ax is None:
		fig=matplotlib.pyplot.figure(figsize=(16, 16))
	if ax is None:
		ax=fig.add_subplot()
	# ax.invert_yaxis()

	posei=env.pose_history[:, env.N, :]
	for i in range(env.N):
					posei=env.pose_history[:, i, :]
					deadi=env.dead_history[:, i]
					posei=posei[~deadi]
					ax.plot(posei[:,0], posei[:, 1], label='Agent '+str(i), color='aqua')
	ax.plot(posei[:env.t-1,0], posei[:env.t-1, 1], linestyle='dashdot',  label='Virtual Leader', color='blue', zorder=3)
	line=[None]*env.N

	for wall in env.walls:
					ax.add_patch(Rectangle((wall.from_x, wall.from_y), wall.length_x, wall.length_y, color='black'))
	ax.scatter(env.xL0, env.yL0, marker='*', zorder=3, color='blue', s=150)
	ax.scatter(env.xG, env.yG, marker='*', zorder=3, color='blue', s=150)
	for agent in env.agents:
					ax.scatter(env.xL0+agent.dx, env.yL0+agent.dy, color='limegreen')
					ax.scatter(env.pose_history[env.t-1, agent.id, 0], env.pose_history[env.t-1, agent.id, 1], color='red' if agent.is_dead else 'limegreen')
	return fig


In [33]:
directory = '../run_exp_10'
if not os.path.exists(directory):
	os.makedirs(directory)
df=pandas.DataFrame(columns=['id','sensor_range' , 'n1', 'n2', 'n3', 'N', 'lived_agents', 'leader_goal_distance', 't', 'w1', 'w2', 'w3' ])
df.set_index('id', inplace=True)

In [34]:
n=10

In [35]:
DX,DY=calcDF(n)
env = Env(
        width=ENV_SIZE, height=ENV_SIZE, goal_x=GOAL_X, goal_y=GOAL_Y, N=n ** 2,
        desired_X=DX, desired_Y=DY, sensor_range=SENSOR_RANGE, leader_x=XL, leader_y=YL, robot_radius=ROBOT_RADIUS,
        sensor_detection_count=SENSOR_DETECTION_COUNT, buffer_size=MAX_T
    )
env.addObstacle(400, 400, 600, 600)


In [36]:
episode_gui(env, 8.5, 1, 1)

In [37]:
# alive_agent_count(env)

In [38]:
n_max=50
if not os.path.exists(directory):
	os.makedirs(directory)
df=pandas.DataFrame(columns=['id','sensor_range' , 'n1', 'n2', 'n3', 'N', 'lived_agents', 'leader_goal_distance', 't', 'w1', 'w2', 'w3' ])
df.set_index('id', inplace=True)

In [39]:
# file_name=os.path.join(directory, 'var.npz')
n_max=50
directory = '../run_exp_12'
if not os.path.exists(directory):
	os.makedirs(directory)

csv_file_name=os.path.join(directory, 'report.csv')
if os.path.exists(csv_file_name):
	df=pandas.read_csv(csv_file_name, header=None, names=['id','sensor_range' , 'n1', 'n2', 'n3', 'N', 'lived_agents', 'leader_goal_distance', 't', 'w1', 'w2', 'w3'])
	run, _, n01, n02, n03, n00, _, _, _, _, _, _=list(df.iloc[-1])
	# cur_state=numpy.load(file_name)
	# n03=int(n03)-1
	# n00=int(n00)
	# n01=int(n01)
	n02=int(n02)-1
	df.set_index('id', inplace=True)
else:
    # killed_list=[]
    # n01=n_max
    n02 = n_max
    # n03 = n_max
    run=0
    # n00=2

fig=plt.Figure()
ax=fig.add_subplot()


In [40]:
n=10
w1=5
w3=1
n1=50
n3=10
for n2 in range(n02, -1, -1):
				run=run+1
				w2=n2/10
				print('w3=',w3)
				print('run='+str(run))
				ep_file=os.path.join(directory, str(run)+'_'+str(n)+'_'+str(w1)+'_'+str(w2)+'_'+str(w3))
				#'id','sensor_range' , 'w1', 'w2', 'w3', 'N', 'lived_agents', 'leader_goal_distance', 't', 'v_mean'
				env.episode(w1, w2, w3)
				# env.save_episode(ep_file)

				row=[SENSOR_RANGE, n1, n2, n3, n, alive_agent_count(env), leader_goal_distance(env), env.t, w1, w2, w3]
				df.loc[run]=row
				print(row)
						# env.episode(w1/10, w2/10, w3/10, killed_list)
						# env.save_episode(ep_file+'a')

				# numpy.savez(file_name, run=run, n1=n1, n2=n2, n3=n3, n=n)

				plot_trajectory(env, ax=ax)
				plt_file=ep_file+'.jpg'
				fig.savefig(plt_file)
				ax.clear()
				# df.iloc[-1].to_csv(csv, mode='a', index=False, header=False)
				# run=run+1
				with open(csv_file_name, 'a',newline='' ) as f:
					writer=csv.writer(f)
					writer.writerow([run]+row)

				# run=run+1
				# df.to_csv()

n02=n_max

In [41]:
df[df['t']==50000]

,sensor_range,n1,n2,n3,N,lived_agents,leader_goal_distance,t,w1,w2,w3
id,,,,,,,,,,,
42.0,10,50,9,10,10,64,0.254727,50000,5,0.9,1
44.0,10,50,7,10,10,72,0.238505,50000,5,0.7,1
46.0,10,50,5,10,10,46,0.470753,50000,5,0.5,1
48.0,10,50,3,10,10,48,0.363492,50000,5,0.3,1
49.0,10,50,2,10,10,38,0.205498,50000,5,0.2,1
50.0,10,50,1,10,10,45,0.060098,50000,5,0.1,1


In [ ]:
 # file_name=os.path.join(directory, 'var.npz')
n_max=100
directory = '../run_exp_11'
if not os.path.exists(directory):
	os.makedirs(directory)

csv_file_name = os.path.join(directory, 'report.csv')
if os.path.exists(csv_file_name):
	df = pandas.read_csv(csv_file_name, header=None,
						 names=['id', 'sensor_range', 'n1', 'n2', 'n3', 'N', 'lived_agents', 'leader_goal_distance',
								't', 'w1', 'w2', 'w3'])
	run, _, n01, n02, n03, n00, _, _, _, _, _, _ = list(df.iloc[-1])
	# cur_state=numpy.load(file_name)
	# n03=int(n03)-1
	# n00=int(n00)
	# n01=int(n01)
	n01 = int(n01) - 1
	df.set_index('id', inplace=True)
else:
	# killed_list=[]
	# n01=n_max
	n01 = n_max
	# n03 = n_max
	run = 0
# n00=2

fig = plt.Figure()
ax = fig.add_subplot()

n = 10
w2 = 1
w3 = 1
n2 = 10
n3 = 10
for n1 in range(n01, -1, -1):
	run = run + 1
	w1 = n1 / 10
	print('w1=', w1)
	print('run=' + str(run))
	ep_file = os.path.join(directory, str(run) + '_' + str(n) + '_' + str(w1) + '_' + str(w2) + '_' + str(w3))
	#'id','sensor_range' , 'w1', 'w2', 'w3', 'N', 'lived_agents', 'leader_goal_distance', 't', 'v_mean'
	env.episode(w1, w2, w3)
	# env.save_episode(ep_file)

	row = [SENSOR_RANGE, n1, n2, n3, n, alive_agent_count(env), leader_goal_distance(env), env.t, w1, w2, w3]
	df.loc[run] = row
	print(row)
	# env.episode(w1/10, w2/10, w3/10, killed_list)
	# env.save_episode(ep_file+'a')

	# numpy.savez(file_name, run=run, n1=n1, n2=n2, n3=n3, n=n)

	plot_trajectory(env, ax=ax)
	plt_file = ep_file + '.jpg'
	fig.savefig(plt_file)
	ax.clear()
	# df.iloc[-1].to_csv(csv, mode='a', index=False, header=False)
	# run=run+1
	with open(csv_file_name, 'a', newline='') as f:
		writer = csv.writer(f)
		writer.writerow([run] + row)

# run=run+1
# df.to_csv()

n01 = n_max

w1= 8.5
run=16.0


In [ ]:
# file_name=os.path.join(directory, 'var.npz')
n_max=100
directory = '../run_exp_13'
if not os.path.exists(directory):
	os.makedirs(directory)

csv_file_name = os.path.join(directory, 'report.csv')
if os.path.exists(csv_file_name):
	df = pandas.read_csv(csv_file_name, header=None,
						 names=['id', 'sensor_range', 'n1', 'n2', 'n3', 'N', 'lived_agents', 'leader_goal_distance',
								't', 'w1', 'w2', 'w3'])
	run, _, n01, n02, n03, n00, _, _, _, _, _, _ = list(df.iloc[-1])
	# cur_state=numpy.load(file_name)
	# n03=int(n03)-1
	# n00=int(n00)
	# n01=int(n01)
	n03 = int(n03) - 1
	df.set_index('id', inplace=True)
else:
	# killed_list=[]
	# n01=n_max
	n03 = n_max
	# n03 = n_max
	run = 0
# n00=2

fig = plt.Figure()
ax = fig.add_subplot()

n = 10
w1 = 5
w2 = 5
n1 = 50
n2 = 50
for n3 in range(n03, -1, -1):
	run = run + 1
	w3 = n3 / 10
	print('w3=', w3)
	print('run=' + str(run))
	ep_file = os.path.join(directory, str(run) + '_' + str(n) + '_' + str(w1) + '_' + str(w2) + '_' + str(w3))
	#'id','sensor_range' , 'w1', 'w2', 'w3', 'N', 'lived_agents', 'leader_goal_distance', 't', 'v_mean'
	env.episode(w1, w2, w3)
	# env.save_episode(ep_file)

	row = [SENSOR_RANGE, n1, n2, n3, n, alive_agent_count(env), leader_goal_distance(env), env.t, w1, w2, w3]
	df.loc[run] = row
	print(row)
	# env.episode(w1/10, w2/10, w3/10, killed_list)
	# env.save_episode(ep_file+'a')

	# numpy.savez(file_name, run=run, n1=n1, n2=n2, n3=n3, n=n)

	plot_trajectory(env, ax=ax)
	plt_file = ep_file + '.jpg'
	fig.savefig(plt_file)
	ax.clear()
	# df.iloc[-1].to_csv(csv, mode='a', index=False, header=False)
	# run=run+1
	with open(csv_file_name, 'a', newline='') as f:
		writer = csv.writer(f)
		writer.writerow([run] + row)

# run=run+1
# df.to_csv()

n03 = n_max